In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os
import keras
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

Using TensorFlow backend.


In [2]:
def build_lstm_fft_model(hidden_1,
                         dense_1,
                         kernel_initializer='normal',
                         dropout=None,
                         optimizer=None,
                         input_shape=(5, 256)):
    
    model = Sequential()
    model.add(LSTM(hidden_1, return_sequences=False, input_shape=input_shape, name='lstm_1'))
    model.add(BatchNormalization())
    model.add(Dense(dense_1, activation='relu', name='dense_1'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid', name='output'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        
    return model

In [3]:
def get_humans_nonhumans(data_dir, humans=None, nonhumans=None):
    
    nonhumans_path = os.path.join(data_dir, 'Data_win_fft_Nonhuman.npy')
    humans_path = os.path.join(data_dir, 'Data_win_fft_Human.npy')
    n_nonhumans = np.load(nonhumans_path, allow_pickle=True)
    n_humans = np.load(humans_path, allow_pickle=True) 
    
    humans = np.vstack((humans, n_humans)) if humans is not None else n_humans
    nonhumans = np.vstack((nonhumans, n_nonhumans)) if nonhumans is not None else n_nonhumans
        
    labels = [1 for i in range(len(humans))]
    labels += [0 for i in range(len(nonhumans))]
        
    return humans, nonhumans, labels

In [4]:
def get_upstream_data(radar_type, scaling=True):
    
    data_dir = '/scratch/sk7898/deep_radar/data/classification'

    if radar_type:
        data_dir = os.path.join(data_dir, radar_type)
        humans, nonhumans, labels = get_humans_nonhumans(data_dir, humans=None, nonhumans=None)

        data = np.vstack((humans, nonhumans)) 
        labels = np.array(labels)
    else:
        humans = None
        nonhumans=None
        radar_types = ['Austere', 'Bumblebee']
        for radar_type in radar_types:
            radar_data_dir = os.path.join(data_dir, radar_type)
            humans, nonhumans, labels = get_humans_nonhumans(radar_data_dir, 
                                                             humans=humans, 
                                                             nonhumans=nonhumans)

        data = np.vstack((humans, nonhumans)) 
        labels = np.array(labels)

    n_data = data.swapaxes(1, 2)
    data = np.absolute(n_data) #(no_of_windows, 5, 256)
    
    X_train_val, X_test, y_train_val, y_test = train_test_split(data, 
                                                                labels,
                                                                test_size=0.1,
                                                                random_state=42)

    X_train, X_val, y_train, y_val = train_test_split(X_train_val, 
                                                      y_train_val,
                                                      test_size=0.1,
                                                      random_state=42)

    if scaling:
        scaler = preprocessing.StandardScaler().fit(X_train.reshape(X_train.shape[0], -1))
        X_train = scaler.transform(X_train.reshape(X_train.shape[0], -1))
        X_test = scaler.transform(X_test.reshape(X_test.shape[0], -1))
        X_val = scaler.transform(X_val.reshape(X_val.shape[0], -1))

        X_train = X_train.reshape(X_train.shape[0], n_data.shape[1], n_data.shape[2])
        X_test = X_test.reshape(X_test.shape[0], n_data.shape[1], n_data.shape[2])
        X_val = X_val.reshape(X_val.shape[0], n_data.shape[1], n_data.shape[2])
        
    return X_train, y_train, X_val, y_val, X_test, y_test

*Data Statistics*
- Austere: Humans - 15021; Nonhumans - 7170; Total - 22191
- Bumblebee: Humans - 8594; Nonhumans - 23594; Total - 32188 

In [8]:
radar_type = ''
epochs = 10
batch_size = 64
learning_rate = 1e-4
dropout = 0.3
hidden_1 = 128
dense_1 = 64

model_dir = '/scratch/sk7898/pedbike/models/upstream'

if radar_type:
    model_dir = os.path.join(model_dir, radar_type)
    model_path = os.path.join(model_dir, 'best_model.h5')
else:
    model_dir = os.path.join(model_dir, 'Austere_Bumblebee')
    model_path = os.path.join(model_dir, 'best_model.h5') #(44046, 5, 256) (4895, 5, 256) (5438, 5, 256)

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
X_train, y_train, X_val, y_val, X_test, y_test = get_upstream_data(radar_type=radar_type)

optimizer = keras.optimizers.Adam(lr=learning_rate)
model = build_lstm_fft_model(hidden_1,
                             dense_1,
                             dropout=dropout,
                             optimizer=optimizer,
                             input_shape=(5, 256))

H_train = model.fit(x=X_train,
                    y=y_train,
                    validation_data=(X_val, y_val),
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=True)

predictions = model.evaluate(x=X_test, y=y_test)
print(predictions)

model.save(model_path)